In [6]:

import tensorflow as tf
from tensorflow.keras import layers, Model
import cv2
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_image(img_path, target_size=(224, 224)):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)  # Resize the image
    img = img.astype(np.float32) / 255.0
    return img
train_df = pd.read_csv("COMP90086_2023_TLLdataset/train.csv")
test_df = pd.read_csv('COMP90086_2023_TLLdataset/test_candidates.csv')
train_left_images = np.array([load_image(f"COMP90086_2023_TLLdataset/train/left/{img_name}.jpg") for img_name in train_df['left']])
train_right_images = np.array([load_image(f"COMP90086_2023_TLLdataset/train/right/{img_name}.jpg") for img_name in train_df['right']])

datagen = ImageDataGenerator(
    rotation_range=20,         
    width_shift_range=0.2,    
    height_shift_range=0.2,   
    shear_range=0.2,          
    zoom_range=0.2,           
    horizontal_flip=True,     
    fill_mode='nearest'       
)
# Assuming train_left_images and train_right_images are your datasets

def create_pairs(left_images, right_images):
    pairs = []
    labels = []

    for i in range(len(left_images)):
        # Positive pairing
        pairs.append([left_images[i], right_images[i]])
        labels.append(1)

        # Negative pairing
        random_idx = np.random.randint(0, len(left_images))
        while random_idx == i:  # Ensure it's not the same image
            random_idx = np.random.randint(0, len(left_images))
            
        pairs.append([left_images[i], right_images[random_idx]])
        labels.append(0)

    return np.array(pairs), np.array(labels)

pairs, labels = create_pairs(train_left_images, train_right_images)

# Splitting pairs for training
train_pairs = [pairs[:, 0], pairs[:, 1]]

print(train_left_images.shape)
print(train_right_images.shape)








(2000, 224, 224, 3)
(2000, 224, 224, 3)


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def cosine_similarity_tensors(vectors):
    x, y = vectors
    x = tf.keras.backend.l2_normalize(x, axis=-1)
    y = tf.keras.backend.l2_normalize(y, axis=-1)
    return tf.keras.backend.sum(x * y, axis=-1, keepdims=True)

def siamese_with_dense121_and_cosine_similarity(input_shape):
    left_input = layers.Input(input_shape)
    right_input = layers.Input(input_shape)

    # Use DenseNet121 as the base model
    base_model = tf.keras.applications.DenseNet121(weights=None, include_top=False, input_shape=input_shape)
    base_model.load_weights('/Users/yangyusong/Desktop/CV final project/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5')
    for layer in base_model.layers:
        layer.trainable = False

    encoded_left = base_model(left_input)
    encoded_right = base_model(right_input)

    # Flatten the encoded outputs (or you can add more layers if needed)
    encoded_left = layers.Flatten()(encoded_left)
    encoded_right = layers.Flatten()(encoded_right)

    # Use cosine similarity to compute the similarity score
    similarity_score = layers.Lambda(cosine_similarity_tensors)([encoded_left, encoded_right])

    siamese_net = Model(inputs=[left_input, right_input], outputs=similarity_score)
    return siamese_net

input_shape = (224, 224, 3)  # Default input shape for DenseNet121
model_dense121 = siamese_with_dense121_and_cosine_similarity(input_shape)
model_dense121.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Assuming train_left_images and train_right_images are your datasets
left_iterator = datagen.flow(train_left_images, batch_size=32, shuffle=False)
right_iterator = datagen.flow(train_right_images, batch_size=32, shuffle=False)

num_epochs = 5

for epoch in range(num_epochs):
    for batch in range(len(left_iterator)):
        left_batch, left_labels = left_iterator.next()
        right_batch, right_labels = right_iterator.next()        
        # Make sure that your labels are in the correct shape for the batch size
        batch_labels = labels[batch*32 : (batch+1)*32]
        loss = model_dense121.train_on_batch([left_batch, right_batch], batch_labels)



ValueError: too many values to unpack (expected 2)